In [16]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalMaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import h5py

%matplotlib inline

In [17]:
def load_dataset():
    train_dataset = h5py.File('train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [18]:
X_train,y_train,X_test,y_test,classes = load_dataset()

In [19]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [20]:
X_train = X_train/255.
X_test = X_test/255.
y_train = convert_to_one_hot(y_train,6).T
y_test = convert_to_one_hot(y_test,6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [7]:
def sign_model(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3,3))(X_input)
    
    X = Conv2D(1,(8,8),strides=(2,2))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(16,(5,5),strides=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = AveragePooling2D((2,2))(X)
    
    X = Conv2D(16,(5,5),strides=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(120,activation = 'relu')(X)
    X = Dense(84,activation = 'relu')(X)
    X = Dense(6,activation = 'softmax')(X)
    model = Model(inputs = X_input , outputs=X , name='sign_model')
    return model

In [22]:
model = sign_model(X_train.shape[1:])

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(),metrics = ['accuracy'])

In [24]:
model.fit(X_train,y_train,epochs=40,batch_size=50)

Epoch 1/40
22/22 [==============================] - 1s 50ms/step - loss: 1.7897 - accuracy: 0.2444
Epoch 2/40
22/22 [==============================] - 1s 50ms/step - loss: 1.4432 - accuracy: 0.4176
Epoch 3/40
22/22 [==============================] - 1s 50ms/step - loss: 1.1840 - accuracy: 0.5685
Epoch 4/40
22/22 [==============================] - 1s 50ms/step - loss: 0.9731 - accuracy: 0.6352
Epoch 5/40
22/22 [==============================] - 1s 50ms/step - loss: 0.8120 - accuracy: 0.6944
Epoch 6/40
22/22 [==============================] - 1s 50ms/step - loss: 0.7148 - accuracy: 0.7380
Epoch 7/40
22/22 [==============================] - 1s 50ms/step - loss: 0.6473 - accuracy: 0.7704
Epoch 8/40
22/22 [==============================] - 1s 50ms/step - loss: 0.5680 - accuracy: 0.8120
Epoch 9/40
22/22 [==============================] - 1s 50ms/step - loss: 0.5005 - accuracy: 0.8139
Epoch 10/40
22/22 [==============================] - 1s 50ms/step - loss: 0.4927 - accuracy: 0.8222
Epoch 11/

In [25]:
preds = model.evaluate(X_test,y_test,batch_size = 32,verbose = 1,sample_weight = None)
print("Loss:",preds[0])
print("Test Accuracy is:"+str(preds[1]))

4/4 [==============================] - 0s 9ms/step - loss: 0.1946 - accuracy: 0.9417
Loss: 0.1946456879377365
Test Accuracy is:0.9416666626930237


In [26]:
model.summary()

Model: "sign_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 1)         193       
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 1)         4         
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 28, 16)        6